<a href="https://colab.research.google.com/github/mahdi943/InceptionV3-binary/blob/master/Acc75_class3_Training_brain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.io
from numpy import random

import keras
from keras import layers
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, BatchNormalization,Activation, Flatten, GlobalAveragePooling2D, SeparableConv2D
from keras.models import Model

from keras.preprocessing import image
from keras.utils import layer_utils
from keras.layers import Dropout
from keras.regularizers import l2
from keras.regularizers import l1
np.random.seed(5)
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
import cv2
import os
from keras.utils.np_utils import to_categorical
from PIL import Image
from keras.preprocessing.image import load_img,img_to_array,array_to_img,save_img

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data = pd.read_csv('/content/drive/MyDrive/project/dataset-class3/labelTrain-class3.csv') 
data1 = pd.read_csv('/content/drive/MyDrive/project/dataset-class3/labelValid-class3.csv')
data2 = pd.read_csv('/content/drive/MyDrive/project/dataset-class3/labelTest-class3.csv')
data3 = pd.read_csv('/content/drive/MyDrive/project/dataset-class3/labelTestfinal-class3.csv')


In [ ]:
!unzip "/content/drive/MyDrive/project/dataset/Train.zip"
!unzip "/content/drive/MyDrive/project/dataset/Test.zip"
!unzip "/content/drive/MyDrive/project/dataset/Valid.zip"

Archive:  /content/drive/MyDrive/project/dataset/Train.zip
   creating: Train/
  inflating: Train/1.jpg             
  inflating: Train/10.jpg            
  inflating: Train/101.jpg           
  inflating: Train/102.jpg           
  inflating: Train/103.jpg           
  inflating: Train/104.jpg           
  inflating: Train/105.jpg           
  inflating: Train/106.jpg           
  inflating: Train/107.jpg           
  inflating: Train/108.jpg           
  inflating: Train/109.jpg           
  inflating: Train/11.jpg            
  inflating: Train/110.jpg           
  inflating: Train/111.jpg           
  inflating: Train/112.jpg           
  inflating: Train/113.jpg           
  inflating: Train/114.jpg           
  inflating: Train/115.jpg           
  inflating: Train/116.jpg           
  inflating: Train/117.jpg           
  inflating: Train/118.jpg           
  inflating: Train/119.jpg           
  inflating: Train/12.jpg            
  inflating: Train/120.jpg           
  inflati

In [ ]:
y_col = ['Class0', 'Class1']

In [ ]:
from keras.applications import inception_v3
from keras.applications.inception_v3 import InceptionV3

base_model = InceptionV3(weights = 'imagenet', include_top = False, input_shape=(500, 500, 3))
base_model.layers.pop()
base_model.layers.pop()
base_model.layers.pop()
base_model.layers.pop()
for layer in base_model.layers:
  layer.trainable = False
# base_model.summary()

87924736/87910968 [==============================] - 0s 0us/step


In [ ]:
def brain_model(base_model, img_shape = (500,500,3)):
    
    
    # create the input layer (Same as the imageNetv2 input size)
    inputs = tf.keras.Input(shape=img_shape) 
    
    # Forward pass to get the output of the last pooling layer
    X = base_model(inputs)
    #X = Dropout(.5, img_shape)(X)
    X = GlobalAveragePooling2D()(X)
    X = Dropout(.25)(X)
    # Define the new binary classification head 
    
    X = Dense(1024, activation='relu', name='fc1')(X)
    X = Dropout(.5)(X)
    # X = Dense(512, activation='relu', name='fc2')(X)
    # X = Dropout(.25)(X)
    X = Dense(256, activation='relu', name='fc3')(X)
    X = Dropout(.5)(X)
    # X = Dense(128, activation='relu', name='fc4')(X)
    # X = Dropout(.25)(X)
    X = Dense(64, activation='relu', name='fc5')(X)
    X = Dropout(.5)(X)
        
    outputs = Dense(2,activation='softmax', name='predictions')(X)
    model = tf.keras.Model(inputs, outputs)
    
    return model

In [ ]:
#Create your model using the function above

model = brain_model (base_model, img_shape = (500,500,3))

In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 500, 500, 3)]     0         
                                                                 
 inception_v3 (Functional)   (None, 14, 14, 2048)      21802784  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dropout (Dropout)           (None, 2048)              0         
                                                                 
 fc1 (Dense)                 (None, 1024)              2098176   
                                                                 
 dropout_1 (Dropout)         (None, 1024)              0         
                                                             

## 7) Train the Network with  Tuned Parameters

In [ ]:
batch_size = 16
#FOLDER_PATH = Enter the path of the folder that contains images
data_path = "/content/Train" #where you extracted the zip file

#Should augmentations be the same for training and validation? 
#train_aug = ImageDataGenerator(rescale=1./255, ...)
#valid_aug = ImageDataGenerator(rescale=1./255, ...)

#shuffle the data
data = data.sample(frac=1)
data.reset_index(drop = True, inplace = True)

# train_aug = ImageDataGenerator(rescale=1./255, 
#                               #  width_shift_range=[-100,100], 
#                               #  height_shift_range=0.5,       
#                                shear_range=0.2,
#                                horizontal_flip=True,
#                                vertical_flip=True,
#                                rotation_range=90,
#                                brightness_range=[0.2,0.05],
#                                zoom_range=0.2
#                                )

train_aug = ImageDataGenerator(	rescale=1/255.0,	
                               rotation_range=20,	
                               zoom_range=0.05,	
                               width_shift_range=0.1,	
                               height_shift_range=0.1,	
                               shear_range=0.05,	
                               horizontal_flip=True,	
                               vertical_flip=True,	
                               fill_mode="nearest")


valid_datagen = ImageDataGenerator(rescale=1./255)
test_aug = ImageDataGenerator(rescale=1./255)

train_generator = train_aug.flow_from_dataframe(
    dataframe=data,
    directory=data_path,
    x_col='Image',
    y_col= y_col,
    class_mode="raw",
    shuffle=True,
    target_size=(500,500), 
    batch_size=batch_size)




Found 256 validated image filenames.


In [ ]:
valid_generator = test_aug.flow_from_dataframe(
    dataframe=data1,
    directory="/content/Valid",
    x_col='Image',
    y_col= y_col,
    class_mode= "raw",
    target_size=(500,500),
    batch_size=batch_size
)

Found 64 validated image filenames.


In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)
optimizer = tf.keras.optimizers.Adam(lr=0.001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(train_generator, batch_size= 16 ,epochs=25, validation_data = valid_generator, callbacks= callback)
# history = model.fit(train_generator, batch_size= 16 ,epochs=20, validation_data = valid_generator, callbacks= callback)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Epoch 1/25
16/16 [==============================] - 34s 1s/step - loss: 0.9139 - accuracy: 0.6719 - val_loss: 0.4054 - val_accuracy: 1.0000
Epoch 2/25
16/16 [==============================] - 16s 1s/step - loss: 0.6563 - accuracy: 0.7266 - val_loss: 0.4360 - val_accuracy: 0.7812
Epoch 3/25
16/16 [==============================] - 18s 1s/step - loss: 0.4357 - accuracy: 0.8164 - val_loss: 0.0894 - val_accuracy: 1.0000
Epoch 4/25
16/16 [==============================] - 19s 1s/step - loss: 0.2678 - accuracy: 0.9062 - val_loss: 0.0378 - val_accuracy: 1.0000
Epoch 5/25
16/16 [==============================] - 25s 2s/step - loss: 0.1671 - accuracy: 0.9336 - val_loss: 0.0059 - val_accuracy: 1.0000
Epoch 6/25
16/16 [==============================] - 17s 1s/step - loss: 0.1162 - accuracy: 0.9688 - val_loss: 0.0051 - val_accuracy: 1.0000
Epoch 7/25
16/16 [==============================] - 16s 1s/step - loss: 0.0795 - accuracy: 0.9766 - val_loss: 0.0045 - val_accuracy: 1.0000
Epoch 8/25
16/16 [==

In [ ]:
plt.plot(history.history['accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.title('model loss')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.show()

In [ ]:
test_generator = test_aug.flow_from_dataframe(
    dataframe=data2,
    directory="/content/Test",
    x_col='Image',
    y_col= y_col,
    shuffle=True,
    class_mode= "raw",
    target_size=(500,500),
    batch_size=batch_size
)

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn import metrics

y_predict = model.predict(test_generator)

y_predict = y_predict.argmax(-1)
accuracy_score(data3['label'],y_predict)
    
